# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

def query(q):
    cur.execute(q)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    return df

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [6]:
# Your code here
query("""SELECT firstName, lastName
         FROM employees
         JOIN offices
         USING(officeCode)
         WHERE city == 'Boston'""")

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [13]:
# Your code here
query("""SELECT officeCode, COUNT(employeeNumber) AS num_employees
         FROM offices
         LEFT JOIN employees
         USING(officeCode)
         GROUP BY officeCode
         HAVING num_employees = 0""")

,officeCode,num_employees
0,27,0


## Write 3 Questions of your own and answer them

### 1) What are the top 10 products that have made the most money?

In [83]:
# Your code here
query("""SELECT p.productName AS item_name,
         p.MSRP, o.priceEach,
         SUM(quantityOrdered),
         SUM(o.quantityOrdered * o.priceEach) AS total_income
         FROM orderdetails o
         LEFT JOIN products p
         USING(productCode)
         GROUP BY item_name
         ORDER BY total_income DESC
         LIMIT 10""")

,item_name,MSRP,priceEach,SUM(quantityOrdered),total_income
0,1992 Ferrari 360 Spider red,169.34,165.95,1808,276839.98
1,2001 Ferrari Enzo,207.80,205.72,1019,190755.86
2,1952 Alpine Renault 1300,214.30,214.30,961,190017.96
3,2003 Harley-Davidson Eagle Drag Bike,193.66,172.36,985,170686.00
4,1968 Ford Mustang,194.57,165.38,933,161531.48
5,1969 Ford Falcon,173.02,141.88,965,152543.02
6,1980s Black Hawk Helicopter,157.69,134.04,1040,144959.91
7,1998 Chrysler Plymouth Prowler,163.73,135.90,986,142530.63
8,1917 Grand Touring Sedan,170.00,136.00,918,140535.60
9,2002 Suzuki XREO,150.62,122.00,1028,135767.03


### 2) Which orders took the longest to ship?

In [52]:
# Your code here
query("""SELECT o.orderNumber,
         julianday(o.shippedDate) - julianday(o.orderDate) AS days_to_ship
         FROM orders o
         LEFT JOIN orderdetails od
         USING(orderNumber)
         GROUP BY orderNumber
         ORDER BY 2 DESC
         LIMIT 5""")

,orderNumber,days_to_ship
0,10165,65.0
1,10210,8.0
2,10258,8.0
3,10240,7.0
4,10339,7.0


### 3) Which employees served the most customers?

In [53]:
# Your code here
query("""SELECT c.salesRepEmployeeNumber AS employee,
         e.firstName, e.lastName,
         COUNT(c.customerNumber) AS num_customers
         FROM customers c
         JOIN employees e
         ON c.salesRepEmployeeNumber = e.employeeNumber
         GROUP BY 1
         ORDER BY 4 DESC
         LIMIT 5""")

,employee,firstName,lastName,num_customers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1286,Foon Yue,Tseng,7


## Level Up: Display the names of every individual product that each employee has sold

In [72]:
# Your code here
query("""SELECT e.firstName, e.lastName, p.productName
         FROM orders o
         JOIN customers c
         USING(customerNumber)
         JOIN employees e
         ON c.salesRepEmployeeNumber = e.employeeNumber
         JOIN orderdetails od
         USING(orderNumber)
         JOIN products p
         USING(productCode)
         ORDER BY 2""")

,firstName,lastName,productName
0,Loui,Bondur,1962 LanciaA Delta 16V
1,Loui,Bondur,1964 Mercedes Tour Bus
2,Loui,Bondur,1926 Ford Fire Engine
3,Loui,Bondur,1992 Ferrari 360 Spider red
4,Loui,Bondur,1940s Ford truck
...,...,...,...
2991,George,Vanauf,1969 Corvair Monza
2992,George,Vanauf,1969 Ford Falcon
2993,George,Vanauf,1970 Triumph Spitfire
2994,George,Vanauf,1950's Chicago Surface Lines Streetcar


## Level Up: Display the Number of Products each employee has sold

In [89]:
# Your code here
query("""SELECT e.firstName, e.lastName, COUNT(productName) AS num_products_sold, 
         SUM(quantityOrdered) AS total_sold
         FROM orders o
         JOIN customers c
         USING(customerNumber)
         JOIN employees e
         ON c.salesRepEmployeeNumber = e.employeeNumber
         JOIN orderdetails od
         USING(orderNumber)
         JOIN products p
         USING(productCode)
         GROUP BY e.employeeNumber
         ORDER BY 3 DESC""")

,firstName,lastName,num_products_sold,total_sold
0,Gerard,Hernandez,396,14231
1,Leslie,Jennings,331,11854
2,Pamela,Castillo,272,9290
3,Larry,Bott,236,8205
4,Barry,Jones,220,7486
5,George,Vanauf,211,7423
6,Andy,Fixter,185,6246
7,Peter,Marsh,185,6632
8,Loui,Bondur,177,6186
9,Steve,Patterson,152,5561


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!